# **[IntoAI] WEEK3 : Logic**

##### **복잡한 세계의 표현을 형성할 수 있고 추론 과정을 통해서 세계에 대한 새로운 표현들을 유도할 수 있으며 그러한 새 표현들을 이용해서 다음에 할 일을 연역할 수 있는 에이전트를 설게한다.**

### **목차**
1. 논리문 구성하기
2. 지식 베이스 : PropKB & 웜퍼스 세계
3. 지식 기반 에이전트와 추론
4. 분해 증명
5. 순방향과 역방향 연쇄


*인공지능 : 현대적 접근방식(제4판)* 도서를 기반으로 작성되었으며 https://github.com/aimacode/aima-python 을 참고하였습니다.
* **Chapter7 | 논리적 에이전트** 의 내용을 포함하고 있습니다.
* 코드와 이론에 대한 자세한 설명은 (블로그 링크)를 참고하세요.
* **Google Colab**을 기반으로 작동하도록 제작되었습니다.
* notebook을 실행하기 위해서는 **logic.py, utils.py, notebook.py** 모듈이 포함되어야 합니다.


### **0. 기본 환경 세팅**

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 모듈이 들어있는 디렉토리로 이동하기
%cd drive/MyDrive/TA/인공지능 입문/SCS4033_Introduce-to-AI/Week 3

In [ ]:
from utils import *
from logic import *
from notebook import psource

### **1. 논리문 구성하기**

In [ ]:
Symbol('x')
(x, y, P, Q, f) = symbols('x, y, P, Q, f')
# P and not Q
P & ~Q

`Expr` 클래스는 모든 종류의 수학적 표현을 나타내도록 설계되었습니다. 가장 단순한 유형의 `Expr`은 `Symbol` 함수로 정의할 수 있는 기호이며 여러개의 symbol을 한번에 정의할수도 있습니다. 또한 파이썬 infix및 접두사 연산자와 결합하여 사용할 수 있습니다.

이는 `Expr` 클래스가 `&`연사자를 다음과 같은 정의로 오버로드 하기 때문에 작동합니다.
```python
def __and__(self, other): return Expr('&',  self, other)
```

그리고 다른 연산자들에게도 비슷한 오버로드를 수행합니다. `Expr`은 연산자의 경우 항상 문자열인 `op`와 인수의 경우 0개 이상의 식을 가진 튜플인 `args`라는 두 개의 필드를 갖습니다. `expression`은 `Expr`의 인스턴스 또는 숫자를 의미합니다.

In [ ]:
sentence = P & ~Q
print(sentence.op)
print(sentence.args)
print(P.op)
print(P.args)

Pxy = P(x, y)
print(Pxy.op)
print(Pxy.args)

# 중첩 Expr
print(3 * f(x, y) + P(y) / 2 + 1)

`Expr` 클래스는 Propositional Logic sentences의 *논리(logic)*를 정의하는 것이 아니라 expression을 *표현(represent)*하는 방법을 제공한다는 점에 유의해야 합니다. `Expr`을 추상 구문 트리로 생각합니다. `Expr`의 각 인수는 기호, 숫자 또는 내포된 `Expr`일 수 있습니다.

다음은 문장을 구성하는 데 사용할 수 있는 연산자 표입니다.

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

In [ ]:
print(~(P & Q)  |'==>'|  (~P | ~Q))
print(expr('~(P & Q)  ==>  (~P | ~Q)'))     #|'==>'| 안써도 됨
print(expr('sqrt(b ** 2 - 4 * a * c)'))     #pre-define 필요 없음

### 2. 지식 베이스: PropKB & 웜퍼스 세계

#### 2-1 명제 지식베이스 : PropKB

`PropKB` 클래스는 명제 논리 문장의 지식 베이스를 나타내는 데 사용할 수 있습니다.

`KB` 클래스에는 `__init__` 메서드 외에 네 가지 메서드가 있습니다. 여기서 주목할 점은 `ask` 메서드가 단순히 ask_generator 메서드를 호출하기 때문에 직접 지식 베이스 클래스를 만들 때 실제로 구현해야 할 것은 `ask` 함수 자체가 아니라 `ask_generator` 함수일 것입니다.

`init(self, sentence=None)`: 생성자 `__init__`은 하나의 필드 `clauses`를 생성합니다. 이 필드는 지식 베이스의 모든 문장을 나타내는 리스트입니다. 이 중 각각의 문장은 '절(clause)'이 됩니다. 즉, 리터럴(literal)과 논리합(or)만으로 이루어진 문장입니다.

`tell(self, sentence)`: 지식 베이스에 문장을 추가하려면 `tell` 메서드를 사용합니다. 이 메서드는 문장을 가져와 CNF로 변환하고 모든 절을 추출하여 `clauses` 필드에 추가합니다. 따라서 지식 베이스에 절만 추가해야한다는 걱정은 하지 않아도 됩니다. 원하는 형태의 문장을 지식 베이스에 알리고, `tell` 메서드가 CNF로 변환하고 그 결과의 절을 추가하는 일을 처리합니다.

`ask_generator(self, query)`: ask 함수에서 사용되는 `ask_generator` 함수입니다. 이 함수는 `tt_entails` 함수를 호출합니다. 이 함수는 지식 베이스가 `query`를 함의하는 경우 `True`를 반환하고, 그렇지 않은 경우 `False`를 반환합니다. `ask_generator` 함수 자체는 지식 베이스가 `query`를 함의하는 경우 빈 `dict {}`를 반환하고, 그렇지 않은 경우 `None`을 반환합니다. 이렇게 하는 이유는 일차 논리(First-Order Logic)에서 ask_generator 함수가 query를 참으로 만드는 모든 대체 값을 반환해야하기 때문입니다. 따라서 모든 대체 값을 반환하기 위해 dict를 사용하는 것입니다. 대부분 ask 함수를 사용할 것이며, 이 함수는 {} 또는 False를 반환합니다. 그러나 이게 마음에 들지 않으면 True나 False를 반환하는 ask_if_true 함수를 사용할 수 있습니다.

`retract(self, sentence)`: 이 함수는 지식 베이스에서 주어진 문장의 모든 절을 제거합니다. tell 함수와 마찬가지로 지식 베이스에서 제거할 절을 전달할 필요는 없습니다. 어떤 문장이든지 괜찮습니다. 이 함수가 해당 문장을 절로 변환하고 그것을 제거하는 일을 처리합니다.

*CNF란 Conjunctive Normal Form의 약자로, 일반적인 논리식을 일정한 형태로 변환한 것입니다.*

*논리식을 CNF로 변환하면, 논리식을 AND 연산자로 연결된 일련의 절(clause)의 형태로 나타낼 수 있습니다. 이 때, 각 절은 OR 연산자로 연결된 리터럴(literal)의 집합으로 이루어져 있습니다. 즉, CNF는 AND 연산자와 OR 연산자만을 사용하는 논리식으로, 각각의 절은 OR 연산자로 결합되고, 각각의 절 내부의 리터럴들은 AND 연산자로 결합되는 형태를 가지게 됩니다.*

#### 2-2 웜퍼스 세계 지식 베이스

In [ ]:
wumpus_kb = PropKB()
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')
wumpus_kb.tell(~P11)
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)
wumpus_kb.clauses

### 3. 지식 기반 에이전트와 추론

#### 3-1 지식 기반 에이전트

지식 기반 에이전트(Knowledge-based agent)는 지식 베이스를 유지하고 처리하는 간단한 일반 에이전트입니다. 지식 베이스에는 초기에 일부 백그라운드 지식이 포함될 수 있습니다.

KB 에이전트의 목적은 지식 베이스 조작에 대한 추상화 수준을 제공하며, 지식 베이스에서 작동하는 에이전트의 기본 클래스로 사용됩니다.

인지(percept)가 주어지면, KB 에이전트는 인지를 지식 베이스에 추가하고, 최상의 작업에 대해 지식 베이스에 요청한 후, 해당 작업을 수행했다고 지식 베이스에 알리게 됩니다.

우리가 구현한 `KB-Agent`의 구현은 KB 클래스를 상속하는 `KB_AgentProgram` 클래스에 캡슐화되어 있습니다.

In [ ]:
psource(KB_AgentProgram)

#### 3-2 명제 지식 베이스에서의 추론
이번 섹션에서는 문장이 KB에 의해 entail되는지 확인하는 두 가지 알고리즘을 살펴보겠습니다. 우리의 목표는 어떤 문장 $\alpha$에 대해 $\text{KB} \vDash \alpha$가 성립하는지 여부를 결정하는 것입니다.

##### 진리표
모델 체크 접근 방식으로, 이름에서 알 수 있듯이 KB가 참인 모든 가능한 모델을 나열하고 이러한 모델에서 $\alpha$도 참인지 확인합니다. 우리는 KB에서 $n$개의 기호를 나열하고, $2^{n}$개의 모델을 깊이 우선 방식으로 나열하고 KB와 $\alpha$의 진리 여부를 확인합니다.

In [ ]:
psource(tt_check_all)

이 알고리즘은 기본적으로 진리표의 모든 행에 대해 $KB\implies\alpha$의 참값을 계산하고 모든 경우에 참인지 확인합니다.

심볼이 정의된 경우, 이 루틴은 심볼의 모든 진리값 조합을 재귀적으로 구성하고, 그런 다음 해당 모델이 KB와 일치하는지 확인합니다. 주어진 모델은 진리표의 각 행에 해당하며, 이러한 행에 대해 KB 열에 'true'가 있으며 이 열에 대해 쿼리가 참인지 확인합니다.

즉, tt_check_all은 각 모델에 대해 이 논리식을 계산합니다.

`pl_true(kb, model) => pl_true(alpha, model)`

이는 논리적으로 동등한 다음 논리식과 같습니다.

`pl_true(kb, model) & ~pl_true(alpha, model)`

즉, 지식 베이스와 쿼리의 부정이 논리적으로 일관되지 않습니다.

`tt_entails()` 함수는 쿼리에서 심볼을 추출하고 적절한 매개변수로 `tt_check_all()` 함수를 호출합니다.

In [ ]:
psource(tt_entails)

In [ ]:
tt_entails(P & Q, Q)
tt_entails(P | Q, Q)
tt_entails(P | Q, P)
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

In [ ]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)
wumpus_kb.ask_if_true(~P22), wumpus_kb.ask_if_true(P22)

### 4. 분해증명

그러나, resolution을 통한 증명을 구현하는 알고리즘은 복잡한 문장을 처리할 수 없습니다.
따라서, 함의 연산과 이중 함의 연산은 더 단순한 절(clauses)으로 단순화되어야 합니다.
우리는 이미 모든 명제 논리 문장이 절의 연결(disjunction)로 표현 가능함을 알고 있습니다.
이 사실을 우리의 이점으로 활용하기 위해, 입력 문장을 연접 정규형(CNF, Conjunctive Normal Form)으로 단순화할 것입니다. 이는, 문자의 연결로 이루어진 교집합입니다.
예를 들어,
$$(A\lor B)\land (\neg B\lor C\lor\neg D)\land (D\lor\neg E)$$
이는 디지털 전자에서의 합의 곱(Product of sums) 형태와 동일합니다.
<br>
이러한 변환의 개요는 다음과 같습니다.

1. 이중 함의 연산을 함의 연산으로 변환합니다.
<br>
$\alpha\iff\beta$는 $(\alpha\implies\beta)\land(\beta\implies\alpha)$로 작성할 수 있습니다.
<br>
이것은 복합 문장에도 적용됩니다.
<br>
$\alpha\iff(\beta\lor\gamma)$는 $(\alpha\implies(\beta\lor\gamma))\land((\beta\lor\gamma)\implies\alpha)$로 작성할 수 있습니다.
<br>

2. 함의 연산을 논리적으로 등가한 식으로 변환합니다.
<br>
$\alpha\implies\beta$는 $\neg\alpha\lor\beta$로 작성할 수 있습니다.
<br>

3. 부정을 내부로 이동합니다.
<br>
CNF는 원자적인 리터럴만 허용합니다. 그래서 부정은 복합문장에 나타낼 수 없습니다.
De Morgan의 법칙이 여기서 도움이 됩니다.
<br>
$\neg(\alpha\land\beta)\equiv(\neg\alpha\lor\neg\beta)$
<br>
$\neg(\alpha\lor\beta)\equiv(\neg\alpha\land\neg\beta)$
<br>

4. 분배 법칙을 사용하여 합집합과 교집합을 분배합니다.
<br>
합집합과 교집합은 서로 분배될 수 있습니다.
이제 우리는 절의 연결, 합집합 및 부정만 가진 문장을 가지고 있습니다.
따라서 가능한 한 합집합을 분배하여 더 단순한 절의 연결로 이루어진 문장을 얻습니다.
<br>
우리는 다음과 같은 형태를 필요로 합니다.
<br>
$(\alpha_{1}\lor\alpha_{2}\lor\alpha_{3}...)\land(\beta_{1}\lor\beta_{2}\lor\beta_{3}...)\land(\gamma_{1}\lor\gamma_{2}\lor\gamma_{3}...)\land...$

`to_cnf` 함수는 도우미 하위 함수를 사용하여 이 변환을 실행합니다.

In [ ]:
psource(to_cnf)

to_cnf 함수는 세 개의 하위 루틴을 호출합니다.
<br>
eliminate_implications는 바이-임플리케이션과 임플리케이션을 논리적 등가물로 변환합니다.
<br>
move_not_inwards는 합성 명제에서 부정을 제거하고 De Morgan의 법칙을 사용하여 내부로 이동합니다.
<br>
distribute_and_over_or는 합연산자를 교환법칙을 이용하여 분배합니다.

In [ ]:
psource(eliminate_implications)
psource(move_not_inwards)
psource(distribute_and_over_or)

In [ ]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

to_cnf(A |'<=>'| (B & C))

to_cnf(A & (B | (C & D)))

to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

In [ ]:
psource(pl_resolution)
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

### 5. 순방향과 역방향 연쇄

이전에 우리는 KB에 의해 entail되는 문장을 확인하기 위해 두 가지 알고리즘을 살펴볼 것이라고 말했습니다. 여기에 세 번째 알고리즘이 있습니다.
다른 점은 지식 베이스에서 확실한 절만 포함하고 있는 경우에 한해서, 단일 명제 기호 q가 entail되는지를 결정하는 것입니다.
그러나 뭔가 주의해야 할 점이 있습니다. 지식 베이스는 반드시 **Horn 절(clause)**만 포함해야 합니다.
<br>

Horn 절(clause)
Horn 절은 최대한 하나의 양의 리터럴로 구성된 리터럴의 합집합으로 정의할 수 있습니다.
<br>
정확히 하나의 양의 리터럴을 가지는 Horn 절을 *확실한 절(definite clause)*이라고 합니다.
<br>
Horn 절은 다음과 같이 생겼을 수 있습니다.
<br>
$\neg a\lor\neg b\lor\neg c\lor\neg d... \lor z$
<br>
그러나 이 예시 또한 확실한 절입니다.
<br>
De Morgan의 법칙을 이용하면, 위의 예시를 다음과 같이 단순화할 수 있습니다.
<br>
$a\land b\land c\land d ... \implies z$
<br>
이것은 인간이 알고 있는 데이터와 사실을 처리하는 논리적인 표현처럼 보입니다.
감지된(percepts) a, b, c, d ... 가 동시에 참이라고 가정하면, 그 시점에서 z도 참일 것이라는 것을 추론할 수 있습니다.
알고리즘적으로 추론 또는 *해결(resolution)*을 더 쉽게 만드는 Horn 절의 몇 가지 흥미로운 측면이 있습니다.
- 확실한 절은 함축(implication)로 작성될 수 있습니다:
<br>
확실한 절에서 가장 중요한 단순화는 함축으로 작성될 수 있다는 것입니다.
전제(또는 함축에 이르는 지식)는 양의 리터럴의 연결(conjunction)입니다.
결론(시사하는 문장) 또한 양의 리터럴입니다.
문장은 이제 이해하기 쉬워졌습니다.
전제와 결론은 관례적으로 body와 head라고 불립니다.
하나의 양의 리터럴은 fact이라고 합니다.
- Horn 절에서 전방 추론 및 후방 추론을 추론에 사용할 수 있습니다
<br>
전진 연쇄는 검색 알고리즘 장에서 소개한 `AND-OR-Graph-Search`와 의미론적으로 동일합니다. 구현 세부 정보는 곧 설명하겠습니다.
- Horn 절을 사용하여 간단하게 entailment 결정 가능:
<br>
놀랍게도, 전진 및 후진 연쇄 알고리즘은 지식 베이스의 각 요소를 최대 한 번만 탐색하므로 문제를 크게 단순화합니다.
<br>
<br>
함수 `pl_fc_entails`은 전진 연쇄를 구현하여 지식 베이스 `KB`가 심볼 `q`를 entail하는지 확인합니다.
<br>
그러나, `pl_fc_entails`은 보통의 `KB` 인스턴스를 사용하지 않습니다. 
여기서 지식 베이스는 `PropKB` 클래스에서 파생된 `PropDefiniteKB` 클래스의 인스턴스이지만, 수정하여 결정적인 절을 저장하도록 변경되었습니다.
<br>
주요 차이점은 주어진 심볼 `p`를 premise에 가진 KB 절의 목록을 반환하는 `PropDefiniteKB`에 대한 도우미 메소드의 포함입니다.

In [ ]:
psource(PropDefiniteKB.clauses_with_premise)
psource(pl_fc_entails)


이 함수는 입력으로 PropDefiniteKB의 인스턴스인 지식 베이스 KB와 질의 q를 받습니다.

count는 초기에 각 문장의 전제에 있는 기호의 수를 저장합니다.

conjuncts 도우미 함수는 주어진 문장을 연결된 표현식으로 분리합니다.

inferred는 boolean defaultdict로 초기화됩니다. 이후 각 단계에서, 이는 모든 문장의 전제가 유추되었는지 확인하는 데 사용됩니다.

agenda는 초기에 지식 베이스에서 참으로 알려진 문장의 목록을 저장합니다.

is_prop_symbol 도우미 함수는 주어진 기호가 유효한 명제 논리 기호인지 확인합니다.

이제 agenda를 반복하면서 각 반복에서 기호 p를 추출합니다. 질의 q가 p와 동일한 경우, 우리는 직관적으로 entailment가 성립한다는 것을 알 수 있습니다.

문장의 전제가 p인 경우, agenda가 처리되며, count가 1씩 감소합니다. count가 0이 되면 결론이 agenda에 추가됩니다. 이것은 해당 유추의 모든 전제가 참임을 의미합니다.

clauses_with_premise는 PropKB 클래스의 유용한 메소드입니다. 이는 전제에 p가 있는 지식 베이스의 문장 목록을 반환합니다.

In [ ]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))
    
pl_fc_entails(definite_clauses_KB, expr('G'))
pl_fc_entails(definite_clauses_KB, expr('H'))
pl_fc_entails(definite_clauses_KB, expr('I'))
pl_fc_entails(definite_clauses_KB, expr('J'))